In [ ]:
import os
from openai import OpenAI
from typing_extensions import override
from openai import AssistantEventHandler

In [ ]:
# Accessing the keys
# openai_mmaai_key = os.getenv('OPENAI_MMAAI_KEY')
# openai_nflai_key = os.getenv('OPENAI_NFLAI_KEY')

In [ ]:
client = OpenAI()

# Code Interpreter Agent

In [ ]:
# fighter1 = 'Michael Page'
# fighter2 = 'Ian Garry'
fighter1 = 'Brian Ortega'
fighter2 = 'Diego Lopes'

message = f'{fighter1} versus {fighter2}'
# message = f'''
# Generate a detailed prediction for the upcoming fight between {fighter1} and 
# {fighter2} with a detailed explanation of your reasoning behind it.
# '''
# message = f'''
# Research detailed statistics and historical performances of {fighter1} and {fighter2} using the CSV file data,
# and generate a detailed prediction for the fight outcome with a thorough explanation of your reasoning.
# '''
# message = input("Waiting for question: ")

In [ ]:
# assistant_id = "your_existing_assistant_id"

# OR

file1 = client.files.create(
  file=open("./data/master.csv", "rb"),
  purpose='assistants'
)

file2 = client.files.create(
    file=open("./data/fighter_info.csv", "rb"),
    purpose='assistants'
)

assistant = client.beta.assistants.create(
    name="UFC Fight Analyst",
    instructions="""
    You are an expert handicapper for UFC/MMA fights at a major sportsbook in Las Vegas. 
    You have a huge amount and all the historical data you need in master.csv and fighter_info.csv,
    analyze it all and use it to help predict the outcomes of upcoming fights.
    Your analysis should consider factors such as the fighters' past performance, weight class, 
    method of victory, and other professional statistics and methods. 
    Give your final fight prediction at the end, and you always must include in it the winner, 
    method/time of victory, and detailed explanation why you think that outcome is likely.
    """,
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
            "file_ids": [file1.id, file2.id]
        }
    }
)

In [ ]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=message
)

class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

In [ ]:
message = input("Follow up question? ")

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=message
)

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

In [ ]:
message = input("Follow up question? ")
message3 = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=message
)

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

# Basic Math Tutor Example Code

In [ ]:
assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

In [ ]:
# without class stuff ^

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

In [ ]:
# More template assistants code

# Upload a file with an "assistants" purpose
# file = client.files.create(
#   file=open("./data/master.csv", "rb"),
#   purpose='assistants'
# )

# Create an assistant using the file ID
# assistant = client.beta.assistants.create(
#   name="Math Tutor",
#   instructions="You are a personal math tutor. When asked a math question, write and run code to answer the question.",
#   model="gpt-4o",
#   tools=[{"type": "code_interpreter"}],
#   tool_resources={
#     "code_interpreter": {
#       "file_ids": [file.id]
#     }
#   }
# )

# thread = client.beta.threads.create(
#   messages=[
#     {
#       "role": "user",
#       "content": "I need to solve the equation `3x + 11 = 14`. Can you help me?",
#       "attachments": [
#         {
#           "file_id": file.id,
#           "tools": [{"type": "code_interpreter"}]
#         }
#       ]
#     }
#   ]
# )

# image_data = client.files.content("file-abc123")
# image_data_bytes = image_data.read()

# with open("./my-image.png", "wb") as file:
#     file.write(image_data_bytes)
    
# run_steps = client.beta.threads.runs.steps.list(
#   thread_id=thread.id,
#   run_id=run.id
# )